In [2]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import src.models.helpers as mh

import pandas as pd
import numpy as np

In [3]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [54]:
df[df.ID == "405"]

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%)
33763,405,2020-12-22,2.52,97,1.15,2.52,1.15,Male,179.0,39,4.316695,97.042483,58.377991,58.377991,99.956223,45.634921,50,47,22,22
33764,405,2020-12-24,2.53,97,1.25,2.53,1.25,Male,179.0,39,4.316695,97.042483,58.609650,58.609650,99.956223,49.407115,50,47,24,24
33765,405,2020-12-25,2.57,97,1.16,2.57,1.16,Male,179.0,39,4.316695,97.042483,59.536285,59.536285,99.956223,45.136187,51,47,22,22
33766,405,2020-12-26,2.56,96,1.33,2.56,1.33,Male,179.0,39,4.316695,97.042483,59.304626,59.304626,98.925746,51.953125,51,46,25,25
33767,405,2020-12-27,2.55,97,1.40,2.55,1.40,Male,179.0,39,4.316695,97.042483,59.072968,59.072968,99.956223,54.901961,51,47,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34793,405,2023-11-09,2.55,95,1.36,2.55,1.36,Male,179.0,39,4.316695,97.042483,59.072968,59.072968,97.895270,53.333333,51,45,26,26
34794,405,2023-11-10,2.54,96,1.35,2.54,1.35,Male,179.0,39,4.316695,97.042483,58.841309,58.841309,98.925746,53.149606,50,46,26,26
34795,405,2023-11-11,2.63,93,1.31,2.63,1.31,Male,179.0,39,4.316695,97.042483,60.926237,60.926237,95.834317,49.809886,52,43,24,24
34796,405,2023-11-12,2.49,92,1.32,2.49,1.32,Male,179.0,39,4.316695,97.042483,57.683015,57.683015,94.803840,53.012048,49,42,26,26


In [81]:
# Given an df_for_ID, find the longest series of consecutive measurements


def find_longest_consec_series(df_for_ID):

    df_for_ID = df_for_ID.reset_index(drop=True)
    df_for_ID["Prev day"] = df_for_ID["Date Recorded"].shift(1)
    df_for_ID["Days elapsed"] = df_for_ID["Date Recorded"] - df_for_ID["Prev day"]

    # Get first idx where Days elapsed is greater than 1
    idx = df_for_ID[df_for_ID["Days elapsed"] > pd.Timedelta(days=3)].index
    # Add the first idx
    idx = idx.insert(0, 0)
    # Add last idx
    idx = idx.insert(len(idx), len(df_for_ID))

    # Make the difference between the idxs
    diff = np.diff(idx)

    # Get the longest series of consecutive measurements
    idx_max_diff = np.argmax(diff)
    start_idx = idx[idx_max_diff]

    end_idx = idx[idx_max_diff + 1]

    df_for_ID[start_idx:end_idx]
    return df_for_ID[start_idx:end_idx]

(845, 20)
Index([0, 286, 294, 298, 618, 621, 628, 756, 766, 845], dtype='int64')
start idx 298
end idx 618


,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%),Prev day,Days elapsed
298,203,2021-09-20,1.90,98,0.62,1.90,1.05,Female,164.0,38,...,60.859190,60.859190,99.86499,32.631579,38,48,16,16,2021-09-16,"4 days, 0:00:00"
299,203,2021-09-21,1.91,98,1.03,1.91,1.03,Female,164.0,38,...,61.179502,61.179502,99.86499,53.926702,38,48,26,26,2021-09-20,"1 day, 0:00:00"
300,203,2021-09-22,1.95,99,1.08,1.95,1.08,Female,164.0,38,...,62.460748,62.460748,100.88402,55.384615,39,49,27,27,2021-09-21,"1 day, 0:00:00"
301,203,2021-09-23,1.91,99,1.00,1.91,1.00,Female,164.0,38,...,61.179502,61.179502,100.88402,52.356021,38,49,26,26,2021-09-22,"1 day, 0:00:00"
302,203,2021-09-24,1.95,99,1.01,1.95,1.01,Female,164.0,38,...,62.460748,62.460748,100.88402,51.794872,39,49,25,25,2021-09-23,"1 day, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,203,2022-12-08,1.95,98,1.10,1.95,1.10,Female,164.0,38,...,62.460748,62.460748,99.86499,56.410256,39,48,28,28,2022-12-07,"1 day, 0:00:00"
614,203,2022-12-09,1.87,99,1.11,1.87,1.11,Female,164.0,38,...,59.898256,59.898256,100.88402,59.358289,37,49,29,29,2022-12-08,"1 day, 0:00:00"
615,203,2022-12-12,1.82,99,1.12,1.82,1.12,Female,164.0,38,...,58.296698,58.296698,100.88402,61.538462,36,49,30,30,2022-12-09,"3 days, 0:00:00"
616,203,2022-12-15,1.86,98,1.00,1.86,1.00,Female,164.0,38,...,59.577944,59.577944,99.86499,53.763441,37,48,26,26,2022-12-12,"3 days, 0:00:00"


In [82]:
df_for_ID.iloc[start_idx : end_idx + 1]
# df[df["ID"] == "272"].iloc[:417].reset_index(drop=True)

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,...,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%),Prev day,Days elapsed
298,203,2021-09-20,1.90,98,0.62,1.90,1.05,Female,164.0,38,...,60.859190,60.859190,99.86499,32.631579,38,48,16,16,2021-09-16,"4 days, 0:00:00"
299,203,2021-09-21,1.91,98,1.03,1.91,1.03,Female,164.0,38,...,61.179502,61.179502,99.86499,53.926702,38,48,26,26,2021-09-20,"1 day, 0:00:00"
300,203,2021-09-22,1.95,99,1.08,1.95,1.08,Female,164.0,38,...,62.460748,62.460748,100.88402,55.384615,39,49,27,27,2021-09-21,"1 day, 0:00:00"
301,203,2021-09-23,1.91,99,1.00,1.91,1.00,Female,164.0,38,...,61.179502,61.179502,100.88402,52.356021,38,49,26,26,2021-09-22,"1 day, 0:00:00"
302,203,2021-09-24,1.95,99,1.01,1.95,1.01,Female,164.0,38,...,62.460748,62.460748,100.88402,51.794872,39,49,25,25,2021-09-23,"1 day, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,203,2022-12-09,1.87,99,1.11,1.87,1.11,Female,164.0,38,...,59.898256,59.898256,100.88402,59.358289,37,49,29,29,2022-12-08,"1 day, 0:00:00"
615,203,2022-12-12,1.82,99,1.12,1.82,1.12,Female,164.0,38,...,58.296698,58.296698,100.88402,61.538462,36,49,30,30,2022-12-09,"3 days, 0:00:00"
616,203,2022-12-15,1.86,98,1.00,1.86,1.00,Female,164.0,38,...,59.577944,59.577944,99.86499,53.763441,37,48,26,26,2022-12-12,"3 days, 0:00:00"
617,203,2022-12-16,1.88,98,1.12,1.88,1.12,Female,164.0,38,...,60.218567,60.218567,99.86499,59.574468,37,48,29,29,2022-12-15,"1 day, 0:00:00"


In [1]:
def get_consec_days(df):
    df = df.sort_values(by="Date Recorded")
    df["Prev day"] = df["Date Recorded"].shift(1)
    # Compute n days between measurements
    df["Days elapsed"] = df["Date Recorded"] - df["Prev day"]
    # Remove nan
    df = df.dropna(subset=["Days elapsed"])

    # Get first idx where Days elapsed is greater than 1
    idx = df[df["Days elapsed"] > pd.Timedelta(days=3)].index
    print(idx)

    return df


df_tmp = df[df.ID == "405"]
df_tmp = get_consec_days(df_tmp)
# Not finished

NameError: name 'df' is not defined